### Learning PyTorch with Examples
(Author: Justin Johnson)

#### 使用Numpy写一个两层的神经网络

In [ ]:
import numpy as np

N, D_in, H, D_out = 64, 1000, 100, 10 # batch size, input dimension, hidden dimension, output dimension

x = np.random.randn(N,D_in)
y = np.random.randn(N,D_out)

w1 = np.random.randn(D_in,H)
w2 = np.random.randn(H, D_out)

learning_rate = 1e-6
for t in range(500):
    h = x.dot(w1)
    h_relu = np.maximum(h,0)
    y_pred = h_relu.dot(w2)
    
    loss = np.square(y_pred - y).sum()
    print(t,loss)
    
    grad_y_pred = 2 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h<0] = 0
    grad_w1 = x.T.dot(grad_h)
    
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

#### 使用tensor写一个两层的神经网络

In [ ]:
import torch
dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64, 1000, 100, 10

x = torch.randn(N, D_in, device = device, dtype = dtype)
y = torch.randn(N,D_out, device = device, dtype = dtype)

w1 = torch.randn(D_in, H, device = device, dtype = dtype)
w2 = torch.randn(H, D_out, device = device, dtype = dtype)

learning_rate = 1e-6
for t in range(500):
    h = x.mm(w1)
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)
    
    loss = (y_pred -y).pow(2).sum().item()
    print(t,loss)
    
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h<0]=0
    grad_w1 = x.t().mm(grad_h)
    
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

#### 使用Tensor + autograd　写一个两层的神经网络

In [ ]:
import torch

dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64,1000,100,10

x = torch.randn(N, D_in, device = device, dtype = dtype)
y = torch.randn(N,D_out, device = device, dtype = dtype)

w1 = torch.randn(D_in, H, device = device, dtype = dtype,requires_grad = True)
w2 = torch.randn(H, D_out, device = device, dtype = dtype, requires_grad = True)

learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    
    print(t, loss.item())
    loss.backward()
   
    
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

#### 自定义自动求导函数

In [ ]:
import torch

class MyReLU(torch.autograd.Function):
    @staticmethod
    def forward(ctx,input):
        ctx.save_for_backward(input)
        return input.clamp(min=0)
    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = grad_output.clone()
        grad_input[input<0] = 0
        return grad_input
dtype = torch.float
device = torch.device("cpu")

N, D_in, H, D_out = 64,1000,100,10

x = torch.randn(N, D_in, device = device, dtype = dtype)
y = torch.randn(N,D_out, device = device, dtype = dtype)

w1 = torch.randn(D_in, H, device = device, dtype = dtype,requires_grad = True)
w2 = torch.randn(H, D_out, device = device, dtype = dtype, requires_grad = True)

learning_rate = 1e-6
for t in range(500):
    relu = MyReLU.apply
    y_pred = relu(x.mm(w1)).mm(w2)
    loss = (y_pred - y).pow(2).sum()
    print(t, loss.item())
    
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        w1.grad.zero_()
        w2.grad.zero_()

#### 基于静态图模型的Tensorflow

In [ ]:
import tensorflow as tf
import numpy as np

N, D_in, H, D_out = 64,1000,100,10
x = tf.placeholder(tf.float32, shape=(None, D_in))
y = tf.placeholder(tf.float32, shape=(None, D_out))

w1 = tf.Variable(tf.random_normal(D_in,H))
w2 = tf.Variable(tf.random_normal(H,D_out))

h = tf.matmul(x,w1)
h_relu = tf.maximum(h,tf.zeros(1))
y_pred = tf.matmul(h_relu,w2)

loss = tf.reduce_sum((y-y_pred)**2.0)
grad_w1, grad_w2 = tf.gradients(loss,[w1,w2])

learning_rate = 1e-6
new_w1 = w1.assign(w1-learning_rate * grad_w1)
new_w2 = w2.assign(w2-learning_rate * grad_w2)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    x_value = np.random.randn(N,D_in)
    y_value = np.random.randn(N,D_out)
    
    for _ in range(500):
        loss_value,_,_ = sess.run([loss,new_w1,new_w2],feed_dict={x:x_value, y:y_value})
        print(loss_value)


#### PyTorch中的torch.nn包

In [ ]:
import torch
N,D_in,H,D_out = 64,1000,100,10
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

model = troch.nn.Sequential(torch.nn.Linear(D_in,H),
                           torch.nn.ReLU(),
                           torch.nn.Linear(H,D_out))
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4
for t range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred,y)
    print(t,loss.item())
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

#### 使用torch.optim包进行优化

In [ ]:
import torch
N,D_in,H,D_out= 64, 1000,100,10
x= torch.randn(N,D_in)
y = torch.randn(N,D_out)
model = torch.nn.Sequential(torch.nn.Linear(D_in,H),
                           torch,nn.ReLU(),
                           torch.nn.Linear(H,D_out))
loss_fn = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(),lr =learning_rate)
for t in range(500):
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    print(t,loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#### 自定义nn模块

In [ ]:
import torch
class TwoLayerNet(torch.nn.Module):
    def __init__(self,D_in,H,D_out):
        super(TwoLayerNet,self).__init__()
        self.linear1 = torch.nn.Linear(D_in,H)
        self.linear2 = torch.nn.Linear(H,D_out)
    
    def forward(self,x):
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        return y_pred
N,D_in,H,D_out = 64,1000,100,10
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)
model = TwoLayerNet(D_in,H,D_out)
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(),lr=1e-4)
for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred,y)
    print(t,loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#### Control Flow + Weight Sharing

In [ ]:
import random
import torch

class DynamicNet(torch.nn.Module):
    def __init__(self, D_in,H,D_out):
        super(DynamicNet,self).__init__()
        self.input_linear = torch.nn.Linear(D_in,H)
        self.middle_linear = torch.nn.Linear(H,H)
        self.output_linear = torch.nn.Linear(H,D_out)
        
    def forward(self,x):
        h_relu = self.input_linear(x).clamp(min=0)
        for _ in range(random.randint(0,3)):
            h_relu = self.middle_linear(h_relu).clamp(min=0)
        y_pred = self.output_linear(h_relu)
        return y_pred

N, D_in,H,D_out = 64, 1000,100,10
x = torch.randn(N,D_in)
y = torch.randn(N,D_out)

model = DynamicNet(D_in,H,D_out)

criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(),lr = 1e-4,momentum=0.90)

for t in range(500):
    y_pred = model(x)
    loss = criterion(y_pred,y)
    print(t,loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    